#### Cards grouped by tab

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import json
import time

# === 1. Setup Selenium ===
driver = webdriver.Chrome()
url = "https://www.eand.com.eg/StaticFiles/portal2/etisalat/pages/plans/elkart_prepaid.html"
driver.get(url)
time.sleep(5)

# === 2. Parse full HTML with BeautifulSoup ===
html = driver.page_source
driver.quit()
soup = BeautifulSoup(html, "html.parser")

# === 3. Extract tab names & their container mappings ===
tab_map = {}
buttons = soup.select("div.tabs-buttons button")
for btn in buttons:
    target = btn.get("data-target")
    text = btn.get_text(strip=True)
    if target == "akwa_tab":
        tab_map["Cards_tab"] = text
    elif target == "flat_tab":
        tab_map["Cards_tab2"] = text
    elif target == "daily_tab":
        tab_map["Cards_tab3"] = text
    elif target == "menu_tab":
        tab_map["Cards_tab4"] = text

# === 4. Loop through each tab container ===
final_data = {}

for container_id, tab_name in tab_map.items():
    container = soup.select_one(f'#{container_id} > div > section > div > div')
    if not container:
        print(f"⚠️ Skipping {container_id} — not found.")
        continue

    cards = container.select("div.item")
    tab_cards = []

    for card in cards:
        try:
            # Plan Name (fallback to "")
            name_tag = card.select_one("h5.plan-name")
            plan_name = name_tag.get_text(strip=True) if name_tag else ""

            # Price
            price_tag = card.select_one("h5.plan-price")
            price = price_tag.get_text(strip=True).replace(" ", "") if price_tag else ""

            # Details as flat key-value (label: value)
            detail_dict = {}
            for li in card.select("li.list-group-item"):
                label_tag = li.select_one("small")
                value_tag = li.select_one("p.fs-16")

                label = label_tag.get_text(strip=True).replace(" ", "") if label_tag else ""
                value = value_tag.get_text(strip=True).replace(" ", "") if value_tag else ""
                if label or value:
                    detail_dict[label] = value

            tab_cards.append({
                "price": price,
                "plan_name": plan_name,
                "details": detail_dict
            })

        except Exception as e:
            print(f"Error parsing a card: {e}")

    final_data[tab_name] = tab_cards

# === 5. Save to JSON ===
with open("cards_grouped_by_tab.json", "w", encoding="utf-8") as f:
    json.dump(final_data, f, ensure_ascii=False, indent=2)

print("Saved data to 'cards_grouped_by_tab.json'")


#### Cards with terms

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import json
import time

# === SETUP ===
driver = webdriver.Chrome()
url = "https://www.eand.com.eg/StaticFiles/portal2/etisalat/pages/plans/elkart_prepaid.html"
driver.get(url)
wait = WebDriverWait(driver, 10)

# === Extract tab buttons and map to container IDs ===
tab_map = {
    "akwa_tab": "Cards_tab",
    "flat_tab": "Cards_tab2",
    "daily_tab": "Cards_tab3",
    "menu_tab": "Cards_tab4"
}

# Get button name text for mapping
tab_titles = {}
buttons = driver.find_elements(By.CSS_SELECTOR, ".tabs-buttons button")
for btn in buttons:
    key = btn.get_attribute("data-target")
    name = btn.text.strip()
    if key in tab_map:
        tab_titles[tab_map[key]] = {
            "btn_element": btn,
            "title": name
        }

final_data = {}

# === LOOP through tabs ===
for container_id, tab_info in tab_titles.items():
    tab_name = tab_info["title"]
    btn = tab_info["btn_element"]

    # Click the tab to activate it and load its terms
    driver.execute_script("arguments[0].click();", btn)
    time.sleep(2)  # Wait for content and terms to update

    # Parse current HTML snapshot
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # ==== Scrape offers ====
    container = soup.select_one(f'#{container_id} > div > section > div > div')
    cards = container.select("div.item") if container else []
    tab_cards = []

    for card in cards:
        try:
            # Plan Name (fallback to "")
            name_tag = card.select_one("h5.plan-name")
            plan_name = name_tag.get_text(strip=True) if name_tag else ""

            # Price
            price_tag = card.select_one("h5.plan-price")
            price = price_tag.get_text(strip=True).replace(" ", "") if price_tag else ""

            # Details as flat key-value
            detail_dict = {}
            for li in card.select("li.list-group-item"):
                label_tag = li.select_one("small")
                value_tag = li.select_one("p.fs-16")

                label = label_tag.get_text(strip=True).replace(" ", "") if label_tag else ""
                value = value_tag.get_text(strip=True).replace(" ", "") if value_tag else ""

                if label or value:
                    detail_dict[label] = value

            tab_cards.append({
                "price": price,
                "plan_name": plan_name,
                "details": detail_dict
            })
        except Exception as e:
            print(f"Error parsing a card in {tab_name}: {e}")

    # ==== Scrape Terms and Conditions ====
    terms = []
    terms_section = soup.select("#Terms .row.text-container.white-readAbout-sec.box-shadow")
    if terms_section:
        for block in terms_section[0].select("div.col-sm-12"):
            text = block.select_one("p")
            if text:
                terms.append(text.get_text(strip=True))

    # ==== Save into final data ====
    final_data[tab_name] = {
        "offers": tab_cards,
        "الشروط والاحكام": terms
    }

# === Export to JSON ===
with open("cards_with_terms.json", "w", encoding="utf-8") as f:
    json.dump(final_data, f, ensure_ascii=False, indent=2)

print("All cards and terms saved to 'cards_with_terms.json'")
driver.quit()


#### Full Page Data

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import json
import time

# === SETUP SELENIUM ===
driver = webdriver.Chrome()
url = "https://www.eand.com.eg/StaticFiles/portal2/etisalat/pages/plans/elkart_prepaid.html"
driver.get(url)
time.sleep(5)

# === EXTRACT PAGE TITLE & DESCRIPTION ===
page_title = driver.find_element(By.XPATH, ".//h1[@class='GESSTwoBold_font fs-25 mb-3 ']").text.strip()
page_description = driver.find_element(By.XPATH, ".//p[@class='fs-14 grey-color ']").text.strip()

# === MAP TAB BUTTONS TO CONTAINER IDs ===
tab_map = {
    "akwa_tab": "Cards_tab",
    "flat_tab": "Cards_tab2",
    "daily_tab": "Cards_tab3",
    "menu_tab": "Cards_tab4"
}

tab_titles = {}
buttons = driver.find_elements(By.CSS_SELECTOR, ".tabs-buttons button")
for btn in buttons:
    key = btn.get_attribute("data-target")
    name = btn.text.strip()
    if key in tab_map:
        tab_titles[tab_map[key]] = {
            "btn_element": btn,
            "title": name
        }

# === COLLECT ALL DATA ===
final_data = {
    "page_title": page_title,
    "page_description": page_description,
    "الانظمة": {}
}

for container_id, tab_info in tab_titles.items():
    tab_name = tab_info["title"]
    btn = tab_info["btn_element"]

    # Click tab and wait for content and terms to load
    driver.execute_script("arguments[0].click();", btn)
    time.sleep(2)

    # Get updated HTML snapshot
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # === EXTRACT OFFERS ===
    container = soup.select_one(f'#{container_id} > div > section > div > div')
    cards = container.select("div.item") if container else []
    tab_cards = []

    for card in cards:
        try:
            plan_name_tag = card.select_one("h5.plan-name")
            plan_name = plan_name_tag.get_text(strip=True) if plan_name_tag else ""

            price_tag = card.select_one("h5.plan-price")
            price = price_tag.get_text(strip=True).replace(" ", "") if price_tag else ""

            details = {}
            for li in card.select("li.list-group-item"):
                label_tag = li.select_one("small")
                value_tag = li.select_one("p.fs-16")

                label = label_tag.get_text(strip=True).replace(" ", "") if label_tag else ""
                value = value_tag.get_text(strip=True).replace(" ", "") if value_tag else ""

                if label or value:
                    details[label] = value

            tab_cards.append({
                "price": price,
                "plan_name": plan_name,
                "details": details
            })

        except Exception as e:
            print(f"Error parsing a card in {tab_name}: {e}")

    # === EXTRACT TERMS ===
    terms = []
    terms_section = soup.select("#Terms .row.text-container.white-readAbout-sec.box-shadow")
    if terms_section:
        for block in terms_section[0].select("div.col-sm-12"):
            text = block.select_one("p")
            if text:
                terms.append(text.get_text(strip=True))

    final_data["الانظمة"][tab_name] = {
        "offers": tab_cards,
        "الشروط والاحكام": terms
    }

# === EXPORT TO JSON ===
with open("akwa_full_page.json", "w", encoding="utf-8") as f:
    json.dump(final_data, f, ensure_ascii=False, indent=2)

print("Data saved to 'akwa_full_page.json'")
driver.quit()
